In [1]:
import os
import tensorflow as tf
import numpy as np
import math
import matplotlib.pyplot as plt
import cv2
import sklearn.metrics as sklm
from data_read import get_wikiArt_data

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Flatten, Dense
from keras.models import Model

Using TensorFlow backend.


In [3]:
from keras import backend as K
img_dim_ordering = 'tf'
K.set_image_dim_ordering(img_dim_ordering)

In [4]:
#To load the data from the files
X_train,Y_train,X_val,Y_val,X_test,Y_test = get_wikiArt_data(0.8,subtract_mean=True)

[0 1 2 3 4 3 1 3 0 3]
<class 'numpy.ndarray'>


In [5]:
#Then we can try to build the models and train the model
def vgg16_transfer_wiki(input_shape,num_classes):
    model_vgg16_conv = VGG16(weights='imagenet', include_top=False)
    model_vgg16_conv.summary()
    
    #Freeze the bottom conv layers
    for layer in model_vgg16_conv.layers:
        layer.trainable = False
        
    #Create your own input format
    keras_input = Input(shape=input_shape, name = 'image_input')
    
    #Use the generated model 
    output_vgg16_conv = model_vgg16_conv(keras_input)
    
    #Add the fully-connected layers 
    x = Flatten(name='flatten')(output_vgg16_conv)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dense(num_classes, activation='softmax', name='predictions')(x)
    
    #Create your own model 
    pretrained_model = Model(inputs=keras_input, outputs=x)
    pretrained_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return pretrained_model
num_classes = 11
input_shape = (224,224,3)
batch_size = 50
num_epoch = 10
model = vgg16_transfer_wiki(input_shape, num_classes)
hist = model.fit(X_train, Y_train,batch_size = batch_size, epochs=num_epoch, validation_data=(X_val, Y_val), verbose=1)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

AttributeError: 'str' object has no attribute 'ndim'

In [ ]:
#To test whether it is due to the input 
